In [2]:
"""
    Hugging face embeddings. This script should be run only once.
"""

import os
import pandas as pd
from haystack.components.embedders import HuggingFaceTEITextEmbedder
from haystack.utils import Secret

data = pd.read_csv('scales.csv', encoding='unicode_escape')
# additional check on the scales converted here
data = data[data['scaleID'].isin(['PID', 'NEO'])]

# if the scales items are language specific, rename the column to 
# 'item' and drop the other language column
if 'item_de' in data.columns:
    data = data.rename(columns={'item_de': 'item'})
    data = data.drop(columns=['item_en'])


ImportError: cannot import name 'Document' from 'haystack' (C:\Users\rober\AppData\Roaming\Python\Python311\site-packages\haystack\__init__.py)

In [ ]:
text_embedder = HuggingFaceTEITextEmbedder(
    model="BAAI/bge-m3", 
    token=Secret.from_token(os.environ['HF_API_KEY'])
)

embeddings = [text_embedder.run(item)['embedding'] for item in data.item]

# save embeddings to file
data['embedding'] = embeddings
data.to_csv('embeddings_BGEM3_de.csv', index=False)


In [ ]:
# multilingual from China
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('intfloat/multilingual-e5-large-instruct')

embeddings = model.encode(data.input.tolist(), convert_to_tensor=False, normalize_embeddings=True)
data['embedding'] = embeddings.tolist()
data.to_csv('embeddings_multilingual_e5_large_instruct_de.csv', index=False)
